In [ ]:
import os
import cv2
import dlib
import numpy as np
import argparse
import inception_resnet_v1
import tensorflow as tf
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import face_recognition
from PIL import Image
import glob
import re
import ntpath
import pickle

In [ ]:
# def _rect_to_css(rect):
#     """
#     Convert a dlib 'rect' object to a plain tuple in (top, right, bottom, left) order
    
#     :Param rect: a dlib 'rect' object
#     :Return: a plain tuple representation of the rect in (top, right, bottom, left) order
#     """
#     return rect.top(), rect.right(), rect.bottom(), rect.left()

In [ ]:
def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
              font_scale=1, thickness=2):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness)

In [ ]:
def main(sess,age,gender,train_mode,images_pl):
    
    # Load array known_face_encoding and known_face_names from npy
    known_face_encodings = np.load("../data/numpy/known_face_encoding.npy")
    known_face_names = np.load("../data/numpy/known_face_names.npy")
    
    # for face detection
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    fa_image = FaceAligner(predictor, desiredFaceWidth=160)
    
    # Initialize some variables
    depth = 16
    k = 8
    frame_number = 0
    face_encodings = []
    face_names = [] # list name in ever frame
    process_this_frame = True
    name = ""
    count = 0
    img_size = 160

    # Capture video
    # Capture from camera extend
#     cap = cv2.VideoCapture(1)

    # Capture from camera of own computer
    cap = cv2.VideoCapture(0)
    
    # Caputure from video offline
#     cap = cv2.VideoCapture('/home/damvantai/Documents/face_recognition_demo/phodibo_hoankiem.mp4')
    
    # Define the codec and create VideoWriter object for write video 
#     fourcc = cv2.VideoWriter_fourcc(*'XVID')
#     out = cv2.VideoWriter('output1.avi',fourcc, 4.0, (640,480))
    
    while True:
    
        number_person = 0
        number_male = 0
        number_female = 0
        
        # Get video frame
        ret, img = cap.read()
        
        # Video no exist
        if not ret:
            print("error: failed to capture image")
            return -1
        
        # Increase ever frame
        frame_number += 1
        
        # Convert the image from BGR color (which opencv user) to RGB color (which face_recognition uses)
        rgb_small_frame = img[:, :, ::-1]
        # input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Convert the image from BGR color to GRAY color
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # img_h, img_w, _ = np.shape(input_img)
        
        # detect faces using dlib detector
        detected = detector(rgb_small_frame, 1)
        faces = np.empty((len(detected), img_size, img_size, 3))
         
        # save frame image
#         if (frame_number % 30 == 0)
#             cv2.imwrite("/home/damvantai/Documents/face_recognition_demo/screenshot/frame%d.jpg" %count, img)
#             count += 1
        
        # ever 10 frame do
        if (frame_number % 10 == 0):
            # Only process every other frame of video to save time
            face_encodings = []
            if process_this_frame:
                
                # Align face 
                image_gray = cv2.cvtColor(rgb_small_frame, cv2.COLOR_RGB2GRAY)
                detected_gray = detector(image_gray, 2)
                
                for i, d in enumerate(detected_gray):
                    image_align = fa_image.align(rgb_small_frame, image_gray, detected_gray[i])
                    face_encodings.append(face_recognition.face_encodings(image_align))
                
                
                # Face encoding ever face from frame 
                face_names = []
                face_encodings = np.asarray(face_encodings)
                for face_encoding in face_encodings:
                    # See if the face is a match for the known face(s)
                    if len(face_encoding) > 0:
                        face_encoding = np.asarray(face_encoding).reshape(128, )
                        matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.42)

                        # If a match was found in known_face_encodings, just use the first one.
                        if True in matches:
                            first_match_index = matches.index(True)
                            name = known_face_names[first_match_index]

                        face_names.append(name)

                        print(np.min(face_recognition.face_distance(known_face_encodings, face_encoding)), name)                        
                        break
                        
                    name = "Unknown"
        
            
        for i, d in enumerate(detected):
            x1, y1, x2, y2= d.left(), d.top(), d.right() + 1, d.bottom() + 1
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            faces[i, :, :, :] = fa_image.align(rgb_small_frame, gray, detected[i])
            
        #
        if len(detected) > 0:
            # predict ages and genders of the detected faces
            ages,genders = sess.run([age, gender], feed_dict={images_pl: faces, train_mode: False})
    
        # draw results
        for i, d in enumerate(detected):
            if genders[i] != 0:
                number_male += 1
            label = "{}, {}".format(int(ages[i]), "F" if genders[i] == 0 else "M")
            draw_label(img, (d.left(), d.top()), label)
        
        # Show number person and male/female
        number_person = len(detected)
        number_female = number_person - number_male
        
         # Display the results
        for d, name in zip(detected, face_names):
#             # Draw a box around the face
#             cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 2)
            
            # Draw a label with a name below the face
            cv2.rectangle(img, (d.left(), d.bottom() - 35), (d.right(), d.bottom()), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(img, name, (d.left() + 6, d.bottom() - 6), font, 1.0, (255, 255, 255), 1)
        
        
        cv2.putText(img, "Male: {}".format(number_male), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(img, "Female: {}".format(number_female), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
#         out.write(img)
        
        cv2.imshow("result", img)
        key = cv2.waitKey(1)

        if key == 27:
            break

In [ ]:
def load_network(model_path):
    sess = tf.Session()
    images_pl = tf.placeholder(tf.float32, shape=[None, 160, 160, 3], name='input_image')
    images_norm = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), images_pl)
    train_mode = tf.placeholder(tf.bool)
    age_logits, gender_logits, _ = inception_resnet_v1.inference(images_norm, keep_probability=0.8,
                                                                 phase_train=train_mode,
                                                                 weight_decay=1e-5)
    gender = tf.argmax(tf.nn.softmax(gender_logits), 1)
    age_ = tf.cast(tf.constant([i for i in range(0, 101)]), tf.float32)
    age = tf.reduce_sum(tf.multiply(tf.nn.softmax(age_logits), age_), axis=1)
    init_op = tf.group(tf.global_variables_initializer(),
                       tf.local_variables_initializer())
    sess.run(init_op)
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(model_path)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print("restore model!")
    else:
        pass
    return sess,age,gender,train_mode,images_pl

In [ ]:
if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--model_path", "--M", default="./models", type=str, help="Model Path")
#     args = parser.parse_args()
    sess, age, gender, train_mode,images_pl = load_network("./models/")
    main(sess,age,gender,train_mode,images_pl)